In [1]:
import pandas as pd

from tqdm import tqdm
from glob import glob

In [2]:
submit = pd.read_csv('./sample_submission.csv', header=0)
submit.columns

Index(['file_name', 'class_id', 'confidence', 'point1_x', 'point1_y',
       'point2_x', 'point2_y', 'point3_x', 'point3_y', 'point4_x', 'point4_y'],
      dtype='object')

In [3]:
width = 1920
height = 1080

# bbox -> 상대좌표계, width, height -> 이미지 해상도
def convert_yolo_to_cv2(width, height, bbox):
    x, y, w, h = bbox
    xmin = int((x - w / 2) * width)
    xmax = int((x + w / 2) * width)
    ymin = int((y - h / 2) * height)
    ymax = int((y + h / 2) * height)
    return xmin, ymin, xmax, ymax

for txt_file in tqdm(glob('./labels/*.txt')):
    # window환경에서 진행하는 경우 split('\\')로 변경 필요
    file_name= txt_file.split('/')[-1].replace('.txt','.png')
    with open(txt_file) as f:
        for ann in f:
            # text결과값의 경우 cls, x, y, w, h, conf 순으로 작업
            cls, x,y,w,h,conf = [float(temp) for temp in ann.split()]
            xmin, ymin, xmax, ymax = convert_yolo_to_cv2(width, height,[x,y,w,h])
            if conf <0.5:
                continue
            # file_name, class, confidence, left, top, right, top, right, bottom, left, bottm
            submit.loc[len(submit)] = [file_name, int(cls), conf, xmin, ymin, xmax, ymin, xmax, ymax, xmin, ymax]


100%|██████████| 3359/3359 [00:03<00:00, 854.05it/s]


In [5]:
submit

,file_name,class_id,confidence,point1_x,point1_y,point2_x,point2_y,point3_x,point3_y,point4_x,point4_y
0,labels\064442001.png,16,0.954003,1139,180,1431,180,1431,494,1139,494
1,labels\064507368.png,16,0.977024,491,124,757,124,757,427,491,427
2,labels\065131036.png,17,0.957297,1108,56,1357,56,1357,317,1108,317
3,labels\065203472.png,1,0.714224,541,762,828,762,828,1039,541,1039
4,labels\065223169.png,16,0.974005,736,160,1015,160,1015,456,736,456
...,...,...,...,...,...,...,...,...,...,...,...
4103,labels\183911746.png,27,0.947210,753,281,1004,281,1004,567,753,567
4104,labels\183911746.png,15,0.885302,511,799,826,799,826,1079,511,1079
4105,labels\183928410.png,27,0.968523,684,49,917,49,917,305,684,305
4106,labels\183928410.png,27,0.599420,1109,75,1353,75,1353,324,1109,324


In [13]:
submit.to_csv('submit2.csv', index=False, encoding='utf-8')